# Download Broadband Seismic Data from IRIS

This notebook downloads broadband seismic waveform data from IRIS for the New Zealand earthquake on **2025-11-06T08:09:54Z**.

## Requirements

```bash
pip install obspy matplotlib numpy
```

In [ ]:
import os
from datetime import datetime, timedelta
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## 1. Configure Download Parameters

In [ ]:
# Event parameters
EVENT_TIME = "2025-11-06T08:09:54Z"
OUTPUT_DIR = "./nz_earthquake_2025-11-06"

# Station selection
NETWORK = "IU,II,G"  # Global seismic networks
STATION = "*"        # All stations
LOCATION = "*"       # All locations
CHANNEL = "BH*"      # Broadband high-gain (BHZ, BHN, BHE)

# Time window
TIME_BEFORE = 300   # 5 minutes before event
TIME_AFTER = 3600   # 1 hour after event

# Geographic selection
MAX_RADIUS = 20.0   # Maximum distance in degrees from epicenter
MIN_RADIUS = 0.0    # Minimum distance in degrees

# Approximate event location (will be updated from catalog)
EVENT_LAT = -37.5
EVENT_LON = 179.0

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Event time: {EVENT_TIME}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Time window: {TIME_BEFORE}s before to {TIME_AFTER}s after event")

## 2. Connect to IRIS and Get Event Information

In [ ]:
# Initialize IRIS client
print("Connecting to IRIS FDSN web services...")
client = Client("IRIS")

# Convert event time
event_time = UTCDateTime(EVENT_TIME)
start_time = event_time - TIME_BEFORE
end_time = event_time + TIME_AFTER

print(f"✓ Connected to IRIS")
print(f"Event time: {event_time}")
print(f"Download window: {start_time} to {end_time}")

In [ ]:
# Search for event in USGS catalog
print("Searching for event in USGS catalog...")

try:
    catalog = client.get_events(
        starttime=event_time - 60,
        endtime=event_time + 60,
        minlatitude=-50,
        maxlatitude=-30,
        minlongitude=165,
        maxlongitude=-175,
    )
    
    if len(catalog) > 0:
        event = catalog[0]
        origin = event.preferred_origin() or event.origins[0]
        magnitude = event.preferred_magnitude() or event.magnitudes[0]
        
        EVENT_LAT = origin.latitude
        EVENT_LON = origin.longitude
        depth = origin.depth / 1000.0
        
        print(f"✓ Found event:")
        print(f"  Origin time: {origin.time}")
        print(f"  Location: {EVENT_LAT:.4f}°N, {EVENT_LON:.4f}°E")
        print(f"  Depth: {depth:.2f} km")
        print(f"  Magnitude: M{magnitude.mag} {magnitude.magnitude_type}")
        
        # Save event
        catalog.write(os.path.join(OUTPUT_DIR, "event_info.xml"), format="QUAKEML")
        print(f"✓ Saved event info")
    else:
        print("⚠ Event not found in catalog")
        event = None
        
except Exception as e:
    print(f"⚠ Error retrieving event: {e}")
    event = None

## 3. Download Waveform Data

In [ ]:
print("Downloading waveform data...")
print(f"  Networks: {NETWORK}")
print(f"  Channels: {CHANNEL}")
print(f"  Radius: {MIN_RADIUS}° to {MAX_RADIUS}° from epicenter")

try:
    stream = client.get_waveforms(
        network=NETWORK,
        station=STATION,
        location=LOCATION,
        channel=CHANNEL,
        starttime=start_time,
        endtime=end_time,
        latitude=EVENT_LAT,
        longitude=EVENT_LON,
        minradius=MIN_RADIUS,
        maxradius=MAX_RADIUS,
    )
    
    print(f"✓ Downloaded {len(stream)} traces")
    print("\nStream summary:")
    print(stream)
    
    # Save waveforms
    waveform_file = os.path.join(OUTPUT_DIR, "waveforms.mseed")
    stream.write(waveform_file, format="MSEED")
    print(f"\n✓ Saved to: {waveform_file}")
    
except Exception as e:
    print(f"✗ Error downloading waveforms: {e}")
    stream = None

## 4. Download Station Metadata

In [ ]:
print("Downloading station metadata...")

try:
    inventory = client.get_stations(
        network=NETWORK,
        station=STATION,
        location=LOCATION,
        channel=CHANNEL,
        starttime=start_time,
        endtime=end_time,
        latitude=EVENT_LAT,
        longitude=EVENT_LON,
        minradius=MIN_RADIUS,
        maxradius=MAX_RADIUS,
        level="response",
    )
    
    n_stations = len(inventory.get_contents()['stations'])
    print(f"✓ Downloaded metadata for {n_stations} stations")
    print("\nInventory summary:")
    print(inventory)
    
    # Save inventory
    inventory_file = os.path.join(OUTPUT_DIR, "station_inventory.xml")
    inventory.write(inventory_file, format="STATIONXML")
    print(f"\n✓ Saved to: {inventory_file}")
    
except Exception as e:
    print(f"✗ Error downloading inventory: {e}")
    inventory = None

## 5. Plot Waveforms

In [ ]:
if stream and len(stream) > 0:
    # Plot all traces
    stream.plot(size=(1000, 800))
    plt.savefig(os.path.join(OUTPUT_DIR, "waveforms_all.png"), dpi=150, bbox_inches='tight')
    print(f"✓ Saved plot to {OUTPUT_DIR}/waveforms_all.png")
else:
    print("No waveforms to plot")

In [ ]:
# Plot first 3 components
if stream and len(stream) >= 3:
    st_subset = stream[:3]
    st_subset.plot(size=(1000, 600))
    plt.savefig(os.path.join(OUTPUT_DIR, "waveforms_3comp.png"), dpi=150, bbox_inches='tight')
    print(f"✓ Saved 3-component plot")

## 6. Basic Waveform Processing Example

In [ ]:
if stream and len(stream) > 0:
    # Create a copy for processing
    st_processed = stream.copy()
    
    # Detrend
    st_processed.detrend('linear')
    
    # Taper edges
    st_processed.taper(max_percentage=0.05)
    
    # Bandpass filter (0.01 - 1.0 Hz for teleseismic signals)
    st_processed.filter('bandpass', freqmin=0.01, freqmax=1.0, corners=4, zerophase=True)
    
    print("✓ Applied processing:")
    print("  - Linear detrend")
    print("  - 5% taper")
    print("  - Bandpass filter: 0.01 - 1.0 Hz")
    
    # Plot processed waveforms
    st_processed.plot(size=(1000, 800))
    plt.savefig(os.path.join(OUTPUT_DIR, "waveforms_processed.png"), dpi=150, bbox_inches='tight')
    print(f"✓ Saved processed waveforms plot")
    
    # Save processed data
    processed_file = os.path.join(OUTPUT_DIR, "waveforms_processed.mseed")
    st_processed.write(processed_file, format="MSEED")
    print(f"✓ Saved processed data to: {processed_file}")

## 7. Summary

In [ ]:
print("="*70)
print("DOWNLOAD COMPLETE")
print("="*70)
print(f"Output directory: {OUTPUT_DIR}")
print("\nFiles created:")
if event:
    print("  ✓ event_info.xml - Event metadata")
if stream:
    print(f"  ✓ waveforms.mseed - {len(stream)} waveform traces")
    print("  ✓ waveforms_processed.mseed - Filtered waveforms")
if inventory:
    print("  ✓ station_inventory.xml - Station metadata with response")
print("\nPlots created:")
print("  ✓ waveforms_all.png")
print("  ✓ waveforms_3comp.png")
print("  ✓ waveforms_processed.png")